In [212]:
def remove_newline(values:list[str]):
    return [line.strip('\n') for line in values]

In [213]:
test_values = []

with open('test.txt') as test_file:
    test_values = remove_newline(test_file.readlines())

test_values

['seeds: 79 14 55 13',
 '',
 'seed-to-soil map:',
 '50 98 2',
 '52 50 48',
 '',
 'soil-to-fertilizer map:',
 '0 15 37',
 '37 52 2',
 '39 0 15',
 '',
 'fertilizer-to-water map:',
 '49 53 8',
 '0 11 42',
 '42 0 7',
 '57 7 4',
 '',
 'water-to-light map:',
 '88 18 7',
 '18 25 70',
 '',
 'light-to-temperature map:',
 '45 77 23',
 '81 45 19',
 '68 64 13',
 '',
 'temperature-to-humidity map:',
 '0 69 1',
 '1 0 69',
 '',
 'humidity-to-location map:',
 '60 56 37',
 '56 93 4']

In [214]:
input_values = []

with open('input.txt') as input_file:
    input_values = remove_newline(input_file.readlines())

input_values

['seeds: 1636419363 608824189 3409451394 227471750 12950548 91466703 1003260108 224873703 440703838 191248477 634347552 275264505 3673953799 67839674 2442763622 237071609 3766524590 426344831 1433781343 153722422',
 '',
 'seed-to-soil map:',
 '2067746708 2321931404 124423068',
 '2774831547 3357841131 95865403',
 '3776553292 3323317283 34523848',
 '4167907733 3453706534 116376261',
 '1190847573 767701596 554806188',
 '2870696950 1975607604 173919437',
 '1980384731 2612856575 87361977',
 '3380570559 2987564153 335753130',
 '3044616387 2451131599 21188806',
 '3909556885 2167390152 154541252',
 '3811077140 2149527041 17863111',
 '4077167815 3804196813 90739918',
 '2528751611 4222771775 72195521',
 '4064098137 3894936731 13069678',
 '4284283994 2700218552 10683302',
 '2468832075 2472320405 59919536',
 '3716323689 3570082795 60229603',
 '1085396685 662250708 105450888',
 '1030174777 1322507784 22912174',
 '1975607604 2446354472 4777127',
 '3828940251 2532239941 80616634',
 '584992388 1930412

# Part 1

In [215]:
from dataclasses import dataclass, field
from copy import deepcopy

@dataclass
class Mapper():
    start_source:int
    end_source:int
    shift:int

    def is_in_range(self, value:int) -> bool:
        return value >= self.start_source and value <= self.end_source
    
    def convert(self, value:int) -> int:
        return value + self.shift
    
    def get_destination_bounds(self) -> tuple[int]:
        return self.start_source + self.shift, self.end_source + self.shift
    
    def backward(self, value) -> int:
        return value - self.shift
    
    def sum(self, other: Mapper) -> dict:
        res = {
            'self_mappers': [],
            'other_mappers': [],
            'overlapping_mapper': None
        }
        
        # Overlapping range
        overlap_start = max(self.start_source, other.start_source - self.shift)
        overlap_end = min(self.end_source, other.end_source - self.shift)

        if overlap_start <= overlap_end:
            # Create a mapper for the overlapping range
            combined_shift = self.shift + other.shift
            res['overlapping_mapper'] = Mapper(overlap_start, overlap_end, combined_shift)

        # Ranges unique to self
        if self.start_source < overlap_start:
            res['self_mappers'].append(Mapper(self.start_source, overlap_start - 1, self.shift))
        if self.end_source > overlap_end:
            res['self_mappers'].append(Mapper(overlap_end + 1, self.end_source, self.shift))
                                       
        # Adjust other's range by self's shift
        other_start_adjusted = other.start_source - self.shift
        other_end_adjusted = other.end_source - self.shift

        # Ranges unique to other
        if other_start_adjusted > self.end_source:
            res['other_mappers'].append(Mapper(overlap_end + 1, other.end_source, other.shift))
        elif other_end_adjusted < self.start_source:
            res['other_mappers'].append(Mapper(other.start_source, overlap_start - 1, other.shift))

        return res

    
@dataclass
class MapperLayer():
    mappers:list[Mapper] = field(default_factory=lambda:list())

    def add(self, mapper:Mapper):
        self.mappers.append(mapper)
        self.mappers.sort(key=lambda x:x.start_source)

    def convert(self, value:int) -> int:
        for mapper in self.mappers:
            if mapper.is_in_range(value):
                return mapper.convert(value)
        return value
    
    def consolidate(self):
        # if there are two adjacent mappers where end[0] == start[1] - 1 and shift[0] == shift[1], then unite them (start = start[0], end = end[1], shift = shift[0])
        is_consolidation_complete = False

        working_values: list[Mapper] = deepcopy(self.mappers)

        while not is_consolidation_complete:
            for idx_0, mapper_0 in enumerate(working_values):
                consolidation_happened = False
                for idx_1, mapper_1 in enumerate(working_values[idx_0+1:]):
                    idx_1 += idx_0 + 1
                    if mapper_0.end_source == mapper_1.start_source - 1 and mapper_0.shift == mapper_1.shift:
                        print(f'mapper consolidation happening {mapper_0}, {mapper_1}')
                        consolidation_happened = True
                        new_mapper = Mapper(
                            mapper_0.start_source,
                            mapper_1.end_source,
                            mapper_0.shift
                        )
                        working_values = [mapper for idx, mapper in enumerate(working_values) if idx not in [idx_0, idx_1]]
                        working_values.append(new_mapper)
                        working_values.sort(key=lambda x:x.start_source)
                        break
                if consolidation_happened:
                    break
                if idx_0 == len(working_values) - 1:
                    is_consolidation_complete = True
        self.mappers = working_values



    def sum(self, other:MapperLayer) -> MapperLayer:
        ending_mappers = MapperLayer()
        source_mappers = sorted(deepcopy(self.mappers), key=lambda x: x.get_destination_bounds()[0])
        destination_mappers = sorted(deepcopy(other.mappers), key=lambda x: x.start_source)

        i = 0

        while i < len(source_mappers):
            overlap_found = False
            j = 0
            while j < len(destination_mappers):
                sum_result = source_mappers[i].sum(destination_mappers[j])
                if sum_result['overlapping_mapper']:
                    print('range overlap found')
                    # add overlapping to ending_mappers
                    ending_mappers.add(sum_result['overlapping_mapper'])

                    # remove elem in position i from source mappers
                    source_mappers = [val for idx,val in enumerate(source_mappers) if idx != i]

                    # add self mappers to source mappers
                    source_mappers = source_mappers + sum_result['self_mappers']

                    # sort source mappers once again
                    source_mappers = sorted(source_mappers, key=lambda x: x.get_destination_bounds()[0])

                    # remove elem in position j from destination mappers
                    destination_mappers = [val for idx, val in enumerate(destination_mappers) if idx != j]

                    # add other mappers to destination mappers
                    destination_mappers = destination_mappers + sum_result['other_mappers']

                    # sort destination mappers once again
                    destination_mappers = sorted(destination_mappers, key=lambda x:x.start_source)

                    # reset i and j - break x2
                    i = -1
                    j = 0
                    break

                j += 1
            
            i += 1
        
        # add source and destination mappers remaining to the ending mappers
        for mapper in source_mappers + destination_mappers:
            ending_mappers.add(mapper)
        # consolidate ending_mappers
        ending_mappers.consolidate()

        return ending_mappers




@dataclass
class MapperLayersList():
    layers:list[MapperLayer] = field(default_factory=lambda: [])

    def convert(self, value:int):
        value_res = value
        for layer in self.layers:
            value_res = layer.convert(value_res)
        return value_res

    def add(self, layer:MapperLayer):
        self.layers.append(layer)
    
    def consolidate(self):
        layer = self.layers[0]
        for layer2 in self.layers[1:]:
            layer = layer.sum(layer2)
        self.layers = [layer]



# TEST MapperLayer Consolidation 

layer = MapperLayer()

layer.add(Mapper(1, 14, 39))
layer.add(Mapper(15, 51, -15))
layer.add(Mapper(52, 53, -15))

layer.consolidate()

assert len(layer.mappers) == 2

# TEST MapperLayersList Consolidation

mapper_layers_list = MapperLayersList()

layer_2 = MapperLayer()

layer_2.add(Mapper(4, 23, 2))
layer_2.add(Mapper(24, 38, 3))
layer_2.add(Mapper(39, 41, -37))

mapper_layers_list.add(layer)
mapper_layers_list.add(layer_2)

consolidated_mapper_layer_list = deepcopy(mapper_layers_list)
consolidated_mapper_layer_list.consolidate()

for seed in range(1, 1000):
    try:
        assert consolidated_mapper_layer_list.convert(seed) == mapper_layers_list.convert(seed)
    except AssertionError as e:
        print(f'Error at seed {seed}', f'consolidated_result {consolidated_mapper_layer_list.convert(seed)}', f'multiple layers result {mapper_layers_list.convert(seed)}')

mapper consolidation happening Mapper(start_source=15, end_source=51, shift=-15), Mapper(start_source=52, end_source=53, shift=-15)
range overlap found
range overlap found
range overlap found


In [216]:
import re
from tqdm import tqdm


REGEX_PATTERN = '(\d+)'

def generate_maps(values:list[str]):
    values = [line for line in values if len(line)>0]

    layers = {
        'seed-to-soil': MapperLayer(),
        'soil-to-fertilizer':MapperLayer(),
        'fertilizer-to-water': MapperLayer(),
        'water-to-light': MapperLayer(),
        'light-to-temperature': MapperLayer(),
        'temperature-to-humidity': MapperLayer(),
        'humidity-to-location': MapperLayer(),
    }

    current_map = None
    for line in tqdm(values, desc='Reading lines to generate maps'):
        if "map" in line:
            current_map = line.split(' ')[0]
        elif current_map is not None:
            line_values = [int(val) for val in re.findall(REGEX_PATTERN, line)]
            layers[current_map].add(
                Mapper(
                    line_values[1], line_values[1] + line_values[2] - 1, line_values[0] - line_values[1]
                )
            )
    return layers

def part_1(values:list[str]):
    maps = generate_maps(values)
    seeds = [int(val) for val in re.findall(REGEX_PATTERN, values[0])]
    locations = []

    for seed in tqdm(seeds, desc="Processing seeds"):
        current_val = seed
        for key, layer in maps.items():
            current_val = layer.convert(current_val)
        locations.append(current_val)
    
    return min(locations)

part_1(test_values)

Processing seeds: 100%|██████████| 4/4 [00:00<00:00, 161319.38it/s]


35

In [217]:
maps = generate_maps(test_values)
layer_list = MapperLayersList()

for layer in maps.values():
    layer.consolidate()
    layer_list.add(layer)

consolidated_layer_list = deepcopy(layer_list)
consolidated_layer_list.consolidate()  

for seed in range(1000):
    try:
        assert(consolidated_layer_list.convert(seed) == layer_list.convert(seed))
    except AssertionError:
        print(f'Seed {seed}: cons {consolidated_layer_list.convert(seed)}, norm {layer_list.convert(seed)}')

Reading lines to generate maps: 100%|██████████| 26/26 [00:00<00:00, 236555.11it/s]

mapper consolidation happening Mapper(start_source=15, end_source=51, shift=-15), Mapper(start_source=52, end_source=53, shift=-15)
range overlap found
range overlap found
range overlap found
range overlap found
range overlap found
range overlap found
range overlap found
range overlap found
range overlap found
range overlap found
range overlap found
range overlap found
Seed 0: cons 39, norm 22
Seed 1: cons 40, norm 23
Seed 2: cons 41, norm 24
Seed 3: cons 42, norm 25
Seed 4: cons 43, norm 26
Seed 5: cons 44, norm 27
Seed 6: cons 48, norm 28
Seed 7: cons 82, norm 29
Seed 8: cons 83, norm 30
Seed 9: cons 84, norm 31
Seed 10: cons 85, norm 32
Seed 11: cons 86, norm 33
Seed 12: cons 87, norm 34
Seed 13: cons 88, norm 35
Seed 14: cons 49, norm 43
Seed 15: cons 15, norm 36
Seed 16: cons 16, norm 37
Seed 17: cons 17, norm 38
Seed 18: cons 18, norm 39
Seed 19: cons 19, norm 40
Seed 20: cons 20, norm 41
Seed 21: cons 21, norm 42
Seed 22: cons 22, norm 90
Seed 23: cons 23, norm 91
Seed 24: cons 

In [218]:
def part_1_v2(values:list[str]):
    maps = generate_maps(values)
    layer_list = MapperLayersList()

    for layer in maps.values():
        layer.consolidate()
        layer_list.add(layer)
        
    layer_list.consolidate()
    print(layer_list)
    
    seeds = [int(val) for val in re.findall(REGEX_PATTERN, values[0])]
    locations = []

    for seed in tqdm(seeds, desc="Processing seeds"):
        current_val = seed
        current_val = layer_list.convert(seed)
        locations.append(current_val)
    
    return min(locations)

part_1_v2(test_values)

Reading lines to generate maps: 100%|██████████| 26/26 [00:00<00:00, 427654.53it/s]


mapper consolidation happening Mapper(start_source=15, end_source=51, shift=-15), Mapper(start_source=52, end_source=53, shift=-15)
range overlap found
range overlap found
range overlap found
range overlap found
range overlap found
range overlap found
range overlap found
range overlap found
range overlap found
range overlap found
range overlap found
range overlap found
MapperLayersList(layers=[MapperLayer(mappers=[Mapper(start_source=0, end_source=5, shift=39), Mapper(start_source=0, end_source=6, shift=42), Mapper(start_source=6, end_source=13, shift=75), Mapper(start_source=7, end_source=10, shift=50), Mapper(start_source=14, end_source=14, shift=35), Mapper(start_source=50, end_source=61, shift=2), Mapper(start_source=62, end_source=62, shift=6), Mapper(start_source=63, end_source=63, shift=-63), Mapper(start_source=64, end_source=74, shift=6), Mapper(start_source=75, end_source=85, shift=-30), Mapper(start_source=86, end_source=97, shift=-26), Mapper(start_source=98, end_source=98,

Processing seeds: 100%|██████████| 4/4 [00:00<00:00, 190650.18it/s]


49

In [219]:
part_1(input_values)

Processing seeds: 100%|██████████| 20/20 [00:00<00:00, 194180.74it/s]


309796150

In [220]:
part_1_v2(input_values)

Reading lines to generate maps: 100%|██████████| 183/183 [00:00<00:00, 595467.52it/s]


mapper consolidation happening Mapper(start_source=662250708, end_source=767701595, shift=423145977), Mapper(start_source=767701596, end_source=1322507783, shift=423145977)
mapper consolidation happening Mapper(start_source=1345419958, end_source=1930412345, shift=-1345419958), Mapper(start_source=1930412346, end_source=1937727385, shift=-1345419958)
mapper consolidation happening Mapper(start_source=94856049, end_source=213050409, shift=-94856049), Mapper(start_source=213050410, end_source=306138731, shift=-94856049)
mapper consolidation happening Mapper(start_source=3106861709, end_source=3415053347, shift=-1237770211), Mapper(start_source=3415053348, end_source=3424691992, shift=-1237770211)
mapper consolidation happening Mapper(start_source=3106861709, end_source=3424691992, shift=-1237770211), Mapper(start_source=3424691993, end_source=3691490076, shift=-1237770211)
mapper consolidation happening Mapper(start_source=3438054298, end_source=3712826168, shift=-691335295), Mapper(star

Processing seeds: 100%|██████████| 20/20 [00:00<00:00, 148998.37it/s]


223287433

# Part 2

In [221]:
def seed_generator(seed_ranges:list[str]):
    seeds = []
    for seed_range in seed_ranges:
        start, length = seed_range.split(' ')
        for seed in range(int(start), int(start)+int(length)):
            yield seed

def get_seed_count(seed_ranges:list[str]) -> int:
    res = 0
    for seed_range in seed_ranges:
        _, length = seed_range.split(' ')
        res += int(length)
    return res

def part_2(values:list[str]):
    REGEX_PATTERN_TWO = '(\d+ \d+)'
    maps = generate_maps(values)
    layer_list = MapperLayersList()

    for layer in maps.values():
        layer.consolidate()
        layer_list.add(layer)
    
    seed_ranges = [val for val in re.findall(REGEX_PATTERN_TWO, values[0])]
    seeds = []
    locations = []

    min_seed = 10**10

    for seed in tqdm(seed_generator(seed_ranges), total=get_seed_count(seed_ranges), desc="Processing seeds"):
        calc = layer_list.convert(seed)
        if calc < min_seed:
            min_seed = calc
                
    return min_seed

part_2(test_values)

Reading lines to generate maps: 100%|██████████| 26/26 [00:00<00:00, 341855.50it/s]


mapper consolidation happening Mapper(start_source=15, end_source=51, shift=-15), Mapper(start_source=52, end_source=53, shift=-15)


Processing seeds: 100%|██████████| 27/27 [00:00<00:00, 414821.27it/s]


46

In [222]:
import concurrent.futures
import re
from tqdm import tqdm

def seed_generator(seed_ranges: list[str]):
    for seed_range in seed_ranges:
        start, length = seed_range.split(' ')
        for seed in range(int(start), int(start) + int(length)):
            yield seed

def get_seed_count(seed_ranges: list[str]) -> int:
    return sum(int(length) for _, length in (seed_range.split(' ') for seed_range in seed_ranges))

def process_seed_range(seed_range: str, layer_list) -> int:
    min_seed = 10**10
    for seed in seed_generator([seed_range]):
        calc = layer_list.convert(seed)
        if calc < min_seed:
            min_seed = calc
    return min_seed

def part_2(values: list[str]):
    REGEX_PATTERN_TWO = '(\d+ \d+)'
    maps = generate_maps(values)  # Assuming this function is defined elsewhere
    layer_list = MapperLayersList()  # Assuming this class is defined elsewhere

    for layer in maps.values():
        layer.consolidate()
        layer_list.add(layer)
    
    seed_ranges = re.findall(REGEX_PATTERN_TWO, values[0])
    min_seeds = []

    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_seed_range, seed_range, layer_list) for seed_range in seed_ranges]
        for future in concurrent.futures.as_completed(futures):
            min_seeds.append(future.result())

    return min(min_seeds)

# Test with some values
test_values = ...  # Define your test values
min_seed = part_2(test_values)
print(min_seed)


TypeError: 'ellipsis' object is not iterable

In [ ]:
part_2(input_values)

Reading lines to generate maps: 100%|██████████| 183/183 [00:00<00:00, 378853.72it/s]


mapper consolidation happening Mapper(start_source=662250708, end_source=767701595, shift=423145977), Mapper(start_source=767701596, end_source=1322507783, shift=423145977)
mapper consolidation happening Mapper(start_source=1345419958, end_source=1930412345, shift=-1345419958), Mapper(start_source=1930412346, end_source=1937727385, shift=-1345419958)
mapper consolidation happening Mapper(start_source=94856049, end_source=213050409, shift=-94856049), Mapper(start_source=213050410, end_source=306138731, shift=-94856049)
mapper consolidation happening Mapper(start_source=3106861709, end_source=3415053347, shift=-1237770211), Mapper(start_source=3415053348, end_source=3424691992, shift=-1237770211)
mapper consolidation happening Mapper(start_source=3106861709, end_source=3424691992, shift=-1237770211), Mapper(start_source=3424691993, end_source=3691490076, shift=-1237770211)
mapper consolidation happening Mapper(start_source=3438054298, end_source=3712826168, shift=-691335295), Mapper(star

Processing seeds:   0%|          | 3991419/2504127863 [00:14<2:32:33, 273141.86it/s]


KeyboardInterrupt: 